In [ ]:
import lightkurve as lk
import pandas as pd
import pickle
import emcee
import corner
import speclite as speclite; from speclite import filters
from tqdm import tqdm
from matplotlib import cm
from matplotlib.artist import Artist
from chromatic import *
from scipy.optimize import minimize
from scipy.optimize import curve_fit
import sys

params = {'legend.fontsize': 'medium',
          'figure.figsize': (6, 4),
         'axes.labelsize': 'large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large'}
plt.rcParams.update(params)
plt.style.use('tableau-colorblind10')

In [ ]:
def initialize_photometry(visit='F21',filt_name = 'gp',dt = 0.05*u.day,gap=0.1):
   
    directory = f'../data/LCO_Photometry/{visit}_phot/'
    data = pd.DataFrame(pd.read_pickle(directory+'AU_Mickle.pkl'))

    
    if visit == 'F21':
        if filt_name == 'gp':
            filt = data.gp
            initial_parameter_guess = [-0.023685, -0.071837, -0.0137700, 0.0113276, 0.0729559, -0.0261908] # these are from an MCMC fit
        if filt_name == 'rp':
            filt = data.rp 
            initial_parameter_guess = [-0.02668624, -0.06327924, -0.011149, -0.01762343, 0.0498755, -0.0297356]
        if filt_name == 'ip':
            filt = data.ip
            initial_parameter_guess = [-0.016888352149918486, -0.03699953137505334, 3.9009679346967575e-05, -0.010208728188986133, 0.02783808342310195, -0.02136799403671536]
    if visit == 'S22':
        if filt_name == 'gp':
            filt = data.gp
            initial_parameter_guess = [-0.032588410440507753, -0.06761059388999908, 0.011023202255339365, -0.057253442026644986, 0.05715036881485087, 0.008102219788961765, 0.05752551767297014]
        if filt_name == 'rp':
            filt = data.rp 
            initial_parameter_guess = [-0.03309734225060938, -0.06690289300817788, 0.011981227215464681, -0.05248070484073764, 0.05568537726412598, 0.007352254284109964, 0.05188364814889117]
        if filt_name == 'ip':
            filt = None
            print('oh no! The S22 visit does not have ip data')

    time = np.array(filt.kb24['time [BJD]'])
    f = filt.kb24['flux']
    flux = np.array(f/np.median(f))
    fluxerr = np.array(filt.kb24['flux_err'])
    t24,f24,e24 = median_bin(time, flux, fluxerr, gap_definition=gap)

    time = np.array(filt.kb26['time [BJD]'])
    f = filt.kb26['flux']
    flux = np.array(f/np.median(f))
    fluxerr = np.array(filt.kb26['flux_err'])
    t26,f26,e26 = median_bin(time,flux,fluxerr,gap_definition=gap)

    time = np.array(filt.kb29['time [BJD]'])
    f = filt.kb29['flux']
    flux = np.array(f/np.median(f))
    fluxerr = np.array(filt.kb29['flux_err'])
    t29,f29,e29 = median_bin(time, flux,fluxerr,gap_definition=gap)

    time = np.array(filt.kb88['time [BJD]'])
    f = filt.kb88['flux']
    flux = np.array(f/np.median(f))
    fluxerr = np.array(filt.kb88['flux_err'])
    t88,f88,e88 = median_bin(time, flux,fluxerr,gap_definition=gap)
    
    if visit == 'F21':
        T0 = (59455.48 + 2400000.5)
        lc24 = lk.LightCurve(time = t24*u.day,flux = f24,
                             flux_err = e24)#.fold(period = 4.865*u.day, epoch_time = T0*u.day)
        lc26 = lk.LightCurve(time = t26*u.day,flux = f26,
                             flux_err = e26)#.fold(period = 4.865*u.day, epoch_time = T0*u.day)
        lc29 = lk.LightCurve(time = t29*u.day,flux = f29,
                             flux_err = e29)#.fold(period = 4.865*u.day, epoch_time = T0*u.day)
        lc88 = lk.LightCurve(time = t88*u.day,flux = f88,
                             flux_err = e88)#.fold(period = 4.865*u.day, epoch_time = T0*u.day)
        lcs = [lc24, lc26, lc29, lc88]
        labels = ['LCO Siding Spring (kb24)', 'LCO CTIO (kb26)', 'LCO Teide (kb29)', 'LCO Siding Spring (kb88)']
        cam_names = ['kb24', 'kb26', 'kb29', 'kb88']
        colors = ['#fac205','#0165fc','#8c000f','#ed0dd9']
    if visit == 'S22':
        T0 = (59455.48 + 2400000.5) + (27.*8.4631427)
        time = np.array(filt.kb87['time [BJD]'])
        f = filt.kb87['flux']
        flux = np.array(f/np.median(f))
        fluxerr = np.array(filt.kb87['flux_err'])
        t87,f87,e87 = median_bin(time, flux,fluxerr,gap_definition=gap)
        lc24 = lk.LightCurve(time = t24*u.day,flux = f24,
                             flux_err = e24)#.fold(period = 4.865*u.day, epoch_time = T0*u.day)
        lc26 = lk.LightCurve(time = t26*u.day,flux = f26,
                             flux_err = e26)#.fold(period = 4.865*u.day, epoch_time = T0*u.day)
        lc29 = lk.LightCurve(time = t29*u.day,flux = f29,
                             flux_err = e29)#.fold(period = 4.865*u.day, epoch_time = T0*u.day)
        lc87 = lk.LightCurve(time = t87*u.day,flux = f87,
                             flux_err = e87)#.fold(period = 4.865*u.day, epoch_time = T0*u.day)
        lc88 = lk.LightCurve(time = t88*u.day,flux = f88,
                             flux_err = e88)#.fold(period = 4.865*u.day, epoch_time = T0*u.day)
        lcs = [lc24, lc26, lc29, lc87, lc88]
        labels = ['LCO Siding Spring kb24', 'LCO CTIO kb26', 'LCO Teide kb29', 'LCO Sutherland kb87', 'LCO Siding Spring kb88']
        cam_names = ['kb24', 'kb26', 'kb29', 'kb87', 'kb88']
        colors = ['#fac205','#0165fc','#8c000f','#10a674','#ed0dd9']

    return lcs, labels, T0, colors, initial_parameter_guess, cam_names

In [ ]:
def sigma_clip(lightcurve, parameters, sigma = 10, k=0, errval = 0.5, **kwargs):
    
    lc = lightcurve
    z_ = lc.flux
    e_ = lc.flux_err
    t_ = lc.time.value
    
    errmask = np.abs(e_) < errval
    z = z_[errmask]
    e = e_[errmask]
    t = t_[errmask]
    
    model = sinusoid_model(t, parameters)+parameters[k+2]
    sigmamask = np.abs(z - model) < sigma*e
    
    clippedtime = t[sigmamask]
    clippedflux = z[sigmamask]
    clippederr = e[sigmamask]
    
    newlc = lk.LightCurve(time = clippedtime*u.day, flux = clippedflux, flux_err = clippederr)

    return newlc

In [ ]:
def sinusoid_model(time, parameters,Period = 4.863):
        
    A = parameters[0]
    B = parameters[1]
    
    argument = (2.0 * np.pi * time)/Period
    
    model = A*np.sin(argument) + B*np.cos(argument) + 1
    
    return model

In [ ]:
def sinusoid_model_curvefit(time, A, B,Period = 4.863):
            
    argument = (2*np.pi*time)/Period
    model = A*np.sin(argument) + B*np.cos(argument) + 1
    
    return model

In [ ]:
def combined_chisq(lightcurves,parameters):
    
    A = parameters[0]
    B = parameters[1]
    offsets = np.array(parameters[2:])
    
    chisq = 0
    ln_like = 0

    for i in range(len(lightcurves)):
        err_weight = np.sum(1/np.sqrt(2*np.pi*(lightcurves[i].flux_err)))
        _model = sinusoid_model(lightcurves[i].time.value, parameters) + offsets[i]
        _chisq = np.sum((lightcurves[i].flux - _model)**2/(lightcurves[i].flux_err)**2)
        chisq+=_chisq
        ln_like += (err_weight - 0.5*_chisq)

    return chisq, ln_like

In [ ]:
def lnprob(parameters,lightcurves, **kwargs): #this will return a total ln_like for all 4 cameras
    
    ln_like = combined_chisq(lightcurves,parameters)[1]
    
    return ln_like

In [ ]:
def lnprob_formcmc(parameters, **kwargs): #this will return a total ln_like for all 4 cameras
    
    ln_like = 0
    
    lightcurves = clipped_lcs

    A = parameters[0]
    B = parameters[1]
    offsets = np.array(parameters[2:])
    
    for i in range(0,ncams):
        model = sinusoid_model(lightcurves[i].time.value,parameters) + offsets[i]
        err_weight = np.sum(1/np.sqrt(2*np.pi*(lightcurves[i].flux_err)))
        chisq = np.sum((lightcurves[i].flux - model)**2/(lightcurves[i].flux_err)**2)
        ln_like += (err_weight - 0.5*chisq)
    
    return ln_like

In [ ]:
def median_bin(time, flux, uncertainty, gap_definition=0.5, visualize=False):
    '''
    Group data into chunks, and median-bin them within those chunks.
    
    Parameters 
    ----------
    time : array 
        The original times, in days. 
    flux : array 
        The original fluxes.   
    uncertainty : array 
        The original uncertainties. 
    gap_definition : float 
        If times are separated by more than this many days,
        consider them separate clumps. A default of 0.5 days 
        would naturally.
    visualize : bool 
        Should we make some visualizations showing details of 
        the binning process or not? 
    
    Returns 
    -------
    binned_time : array 
        The binned times.
    binned_flux : array 
        The binned fluxes. 
    binned_uncertainty : array 
        The binned_fluxes.
    '''
    
    '''
    - Sort the data in time order. '''
    indices = time.argsort(axis=None)
    _f = flux[indices]
    _e = uncertainty[indices]
    _t = time[indices] # _t is now a sorted array of times
    
    '''
    - Identify gap_start as the array indices where np.diff(time) > gap_definition.'''
    gap_start = np.diff(_t) >= gap_definition # this is a list of T/F values

    '''
    - Create an array of clump start times and clump end times. '''
    edges = _t[1:]*gap_start # This replaces all but the 'border' values with 0
    condition = np.where(edges == 0.) # Array of indices
    compressed = np.delete(edges,condition) - 0.5*gap_definition # Removing the values which are not bin edges (which were 0) and offsetting the edge from the points
    _edges = np.insert(compressed,0,(_t[0] - 0.5*gap_definition)) # create a first edge
    bin_edges = np.append(_edges,np.max(_t) + 0.5*gap_definition) # create a last edge
    if visualize:
        plt.figure(figsize=(5,4))
        plt.title('median binning results')
        # plt.xlim(2459455.65,2459455.7)
        plt.errorbar(_t, _f, _e, label='sorted & unbinned', 
                 marker='.', linewidth=0, elinewidth=1, color='gray')
        for i in range(len(bin_edges)):
            plt.axvline(bin_edges[i],color='k')
            if i == 0:
                plt.axvline(bin_edges[i],color='k',label='bin edges')
    
    '''    
    - Create empty arrays (binned_time, binned_flux, binned_uncertainty) with one element per clump.'''
    binned_time = np.array([None]*(len(bin_edges)-1)) # the number of binned times is 1 more than the number of bin_edges
    binned_flux = np.array([None]*(len(bin_edges)-1)) # unless bin_edges includes upper and lower bounds
    binned_uncertainty = np.array([None]*(len(bin_edges)-1))

    # Loop through clumps, select the data points in each clump.
    for i in range(len(bin_edges)-1): 

        start_time = bin_edges[i]
        end_time = bin_edges[i+1]
        def_low = np.array(_t > start_time)
        def_high = np.array(_t < end_time)
        trange = (def_low * def_high)
        
        these_times = _t*trange
        condition = (these_times == 0)
        this_clumps_times = np.delete(these_times, condition)
        these_fluxes = _f*trange
        condition = (these_fluxes == 0)
        this_clumps_fluxes = np.delete(these_fluxes, condition)
        these_errs = _e*trange
        condition = (these_errs == 0)
        this_clumps_uncertainties = np.delete(these_errs, condition)
        
        binned_time[i] = np.median(this_clumps_times) # Set the binned_time for this clump to be the median of the times in the clump.
        binned_flux[i] = np.median(this_clumps_fluxes) # Set the binned_flux for this clump to be median of the fluxes in the clump.
        binned_uncertainty[i] = astropy.stats.mad_std(this_clumps_fluxes)/np.sqrt(len(this_clumps_times)) #  `astropy.stats.mad_std` to be less sensitive than `np.std` to outliers.
        #  # To avoid problems with small numbers of points in a bin, don't let uncertainty be smaller than...
        if binned_uncertainty[i] <= np.median(this_clumps_uncertainties)/np.sqrt(len(this_clumps_times)) :
            binned_uncertainty[i] = np.median(this_clumps_uncertainties)/np.sqrt(len(this_clumps_times))
        if binned_uncertainty[i] <= 0.01:
            binned_uncertainty[i] = 0.01
    
    if visualize:
        plt.errorbar(binned_time, binned_flux, binned_uncertainty,
                     label='binned', linewidth=0,elinewidth=1, color='red',zorder=20)
        plt.legend()
    
    return binned_time, binned_flux, binned_uncertainty

In [ ]:
def median_bin(time, flux, uncertainty, gap_definition=0.5, visualize=False):
    '''
    Group data into chunks, and median-bin them within those chunks.
    
    Parameters 
    ----------
    time : array 
        The original times, in days. 
    flux : array 
        The original fluxes.   
    uncertainty : array 
        The original uncertainties. 
    gap_definition : float 
        If times are separated by more than this many days,
        consider them separate clumps. A default of 0.5 days 
        would naturally.
    visualize : bool 
        Should we make some visualizations showing details of 
        the binning process or not? 
    
    Returns 
    -------
    binned_time : array 
        The binned times.
    binned_flux : array 
        The binned fluxes. 
    binned_uncertainty : array 
        The binned_fluxes.
    '''
    
    '''
    - Sort the data in time order. '''
    indices = time.argsort(axis=None)
    _f = flux[indices]
    _e = uncertainty[indices]
    _t = time[indices] # _t is now a sorted array of times
    
    '''
    - Identify gap_start as the array indices where np.diff(time) > gap_definition.'''
    gap_start = np.diff(_t) >= gap_definition # this is a list of T/F values

    '''
    - Create an array of clump start times and clump end times. '''
    edges = _t[1:]*gap_start # This replaces all but the 'border' values with 0
    condition = np.where(edges == 0.) # Array of indices
    compressed = np.delete(edges,condition) - 0.5*gap_definition # Removing the values which are not bin edges (which were 0) and offsetting the edge from the points
    _edges = np.insert(compressed,0,(_t[0] - 0.5*gap_definition)) # create a first edge
    bin_edges = np.append(_edges,np.max(_t) + 0.5*gap_definition) # create a last edge
    if visualize:
        plt.figure(figsize=(5,4))
        plt.title('median binning results')
        # plt.xlim(2459455.65,2459455.7)
        plt.errorbar(_t, _f, _e, label='sorted & unbinned', 
                 marker='.', linewidth=0, elinewidth=1, color='gray')
        for i in range(len(bin_edges)):
            plt.axvline(bin_edges[i],color='k')
            if i == 0:
                plt.axvline(bin_edges[i],color='k',label='bin edges')
    
    '''    
    - Create empty arrays (binned_time, binned_flux, binned_uncertainty) with one element per clump.'''
    binned_time = np.array([None]*(len(bin_edges)-1)) # the number of binned times is 1 more than the number of bin_edges
    binned_flux = np.array([None]*(len(bin_edges)-1)) # unless bin_edges includes upper and lower bounds
    binned_uncertainty = np.array([None]*(len(bin_edges)-1))

    # Loop through clumps, select the data points in each clump.
    for i in range(len(bin_edges)-1): 
        start_time = bin_edges[i]
        end_time = bin_edges[i+1]
        def_low = np.array(_t > start_time)
        def_high = np.array(_t < end_time)
        trange = (def_low * def_high)
        
        these_times = _t*trange
        condition = (these_times == 0)
        this_clumps_times = np.delete(these_times, condition)
        these_fluxes = _f*trange
        condition = (these_fluxes == 0)
        this_clumps_fluxes = np.delete(these_fluxes, condition)
        these_errs = _e*trange
        condition = (these_errs == 0)
        this_clumps_uncertainties = np.delete(these_errs, condition)
        
        binned_time[i] = np.median(this_clumps_times) # Set the binned_time for this clump to be the median of the times in the clump.
        binned_flux[i] = np.median(this_clumps_fluxes) # Set the binned_flux for this clump to be median of the fluxes in the clump.
        binned_uncertainty[i] = astropy.stats.mad_std(this_clumps_fluxes)/np.sqrt(len(this_clumps_times)) #  `astropy.stats.mad_std` to be less sensitive than `np.std` to outliers.
        #  # To avoid problems with small numbers of points in a bin, don't let uncertainty be smaller than...
        if binned_uncertainty[i] <= np.median(this_clumps_uncertainties)/np.sqrt(len(this_clumps_times)) :
            binned_uncertainty[i] = np.median(this_clumps_uncertainties)/np.sqrt(len(this_clumps_times))
        if binned_uncertainty[i] <= 0.01:
            binned_uncertainty[i] = 0.01
    
    if visualize:
        plt.errorbar(binned_time, binned_flux, binned_uncertainty,
                     label='binned', linewidth=0,elinewidth=1, color='red',zorder=20)
        plt.legend()
    
    return binned_time, binned_flux, binned_uncertainty

In [ ]:
visits = ['F21','S22']
sigmas = [10,5]

for visit in tqdm(visits):
    
    if visit == 'F21':
        filt_names=['gp','rp','ip']
    
    if visit == 'S22':
        filt_names=['gp','rp']
        
    for this_filter in tqdm(filt_names):

        # Set initial parameters for optimization and binning
        lcs, labels, T0, colors, initial_parameter_guess, cam_names = initialize_photometry(visit=visit,filt_name=this_filter,gap=0.01)
        ndim = len(initial_parameter_guess)  
        ncams = ndim - 2
        clipped_lcs = [None]*ncams
        hires_model_times = np.linspace(T0-20,T0+80,10000)

        '''
        PLOT 1 - UNCLIPPED DATA, INITIAL MODEL
        '''
        i = 0
        for lc in lcs:
            # lc.time = lc.time + (T0*u.day) # Re-orienting the time axis from being a phase
            this_model = sinusoid_model(lc.time.value,initial_parameter_guess)+initial_parameter_guess[i+2]
            plt.figure()
            plt.title(f'{visit} {this_filter} {cam_names[i]} binned photometry (PLOT 1)')
            plt.errorbar(lc.time.value,lc.flux,yerr=lc.flux_err,color='red',fmt='o', label = cam_names[i],zorder=-10)
            plt.plot(lc.time.value,this_model,label='initial model',color='blue',zorder=-100) #plot the initial models
            plt.xlim(lc.time.value[0]-5,lc.time.value[-1]+5)
            plt.axvline(T0,label='T0')
            plt.ylim(np.min(lc.flux)-0.05,np.max(lc.flux)+0.05)
            plt.legend(loc='upper right')
            plt.show()
            i += 1  
            
        confirm = input(f"proceed to clip {sigmas[0]}-sigma outliers? y/n")
        if confirm == 'y':
            print("Proceeding")
        if confirm == 'n':
            print("Exiting")
            sys.exit()
        '''
        FIRST ROUND SIGMA CLIP based on the initial fit
        PLOT 2 - UNCLIPPED DATA, ONCE-CLIPPED DATA, INITIAL MODEL
        '''
        i = 0
        for lc in lcs:
            clipped = sigma_clip(lc, initial_parameter_guess,sigma=sigmas[0],k=i)
            hires_model = sinusoid_model(hires_model_times, initial_parameter_guess) + initial_parameter_guess[i+2]
            clipped_lcs[i] = clipped

            plt.figure()
            plt.title(f'{visit} {this_filter} {cam_names[i]} with {sigmas[0]}-sigma outliers clipped (PLOT 2)')
            plt.errorbar(lc.time.value,lc.flux,yerr=lc.flux_err,color='red',fmt='o',label = f'{cam_names[i]} (pre-clip)',zorder=-10)
            plt.errorbar(clipped.time.value,clipped.flux,yerr=clipped.flux_err,color='k',fmt='o',label=f'{cam_names[i]} (post-clip)',zorder=10) #plot the initial models
            plt.plot(hires_model_times,hires_model,label='hires init model',zorder=-15,alpha=1,color='purple') #plot the initial models
            plt.xlim(lc.time.value[0]-5,lc.time.value[-1]+5)
            plt.axvline(T0,label='T0')
            plt.ylim(np.min(lc.flux)-0.05,np.max(lc.flux)+0.05)
            plt.legend(loc='upper right')
            plt.show()
            i += 1
            
        confirm = input(f"Optimize the fit and clip {sigmas[1]}-sigma outliers? y/n")
        if confirm == 'y':
            print("Proceeding")
        if confirm == 'n':
            print("Exiting")
            sys.exit()
        '''
        RUN AN OPTIMIZATION to improve fit parameters
        '''
        def negative_lnprob(parameters, **kwargs):
                return -lnprob(parameters,clipped_lcs)
        function_to_minimize = negative_lnprob
        results = minimize(function_to_minimize, initial_parameter_guess)
        fitted_parameters = results['x']
        offsets = fitted_parameters[2:]
        p_naught = [fitted_parameters[0],fitted_parameters[1]]
        
        '''
        SECOND ROUND SIGMA CLIP based on the optimized fit
        PLOT 3 - UNCLIPPED, TWICE-CLIPPED, and OPTIMIZED MODEL
        '''
        i = 0
        n_obs=0 # this is for the error adjustment 
        for lc in clipped_lcs:
            # optmodel = sinusoid_model(lc.time.value,fitted_parameters) + offsets[i]
            clipped = sigma_clip(lc, fitted_parameters,sigma=sigmas[1],k=i)
            hires_optimized_model = sinusoid_model(hires_model_times, fitted_parameters) + offsets[i]
            clipped_lcs[i] = clipped

            plt.figure()
            plt.title(f'{visit} {this_filter} {cam_names[i]} with {sigmas[1]}-sigma outliers clipped (PLOT 3)')
            plt.errorbar(lcs[i].time.value,lcs[i].flux,yerr=lcs[i].flux_err,color='red',fmt='o',label = f'{cam_names[i]} (pre-clip)',zorder=-10)
            plt.errorbar(clipped.time.value,clipped.flux,yerr=clipped.flux_err,color='k',fmt='o',label=f'{cam_names[i]} (post-clip)',zorder=10) 
            plt.plot(hires_model_times,hires_optimized_model,label='hires optimized model',zorder=-15,alpha=1,color='purple') #plot the initial models
            plt.xlim(lc.time.value[0]-5,lc.time.value[-1]+5)
            plt.axvline(T0,label='T0')
            plt.ylim(np.min(lc.flux)-0.05,np.max(lc.flux)+0.05)
            plt.legend(loc='upper right')
            plt.show()
            n_obs+=len(lc.time)
            i += 1
            
        confirm = input(f"proceed to re-optimize the model and adjust uncertainties? y/n")
        if confirm == 'y':
            print("Proceeding")
        if confirm == 'n':
            print("Exiting")
            sys.exit()
        '''
        RE-OPTIMIZE THE TWICE-CLIPPED DATA
        '''
        newresults = minimize(function_to_minimize, fitted_parameters)
        newfitted_parameters = results['x']
        np.save(f'../data/LCO_Photometry/{visit}_{this_filter}_fittedparameters.npy',newfitted_parameters)
        offsets = newfitted_parameters[2:]
        p_naught = [newfitted_parameters[0],newfitted_parameters[1]]
        '''
        ADJUST UNCERTAINTIES on CLIPPED DATA based on NEW OPTIMIZATION
        PLOT 4 - UNCLIPPED, TWICE-CLIPPED with NEW UNCERTAINTIES, and NEW MODEL
        '''
        i = 0
        for lc in clipped_lcs:
            optimized_model = sinusoid_model(lc.time.value,newfitted_parameters) + offsets[i]
            hires_optimized_model = sinusoid_model(hires_model_times, newfitted_parameters) + offsets[i]
            rchi2 = np.sum(((lc.flux-optimized_model)**2/lc.flux_err**2))/(n_obs-ndim)
            if rchi2 >= 1:
                lc.flux_err = lc.flux_err * np.sqrt(rchi2)            

            plt.figure()
            plt.title(f'{visit} {this_filter} {cam_names[i]} FINAL OPTIMIZED PARAMS (PLOT 4)')
            plt.errorbar(lcs[i].time.value,lcs[i].flux,yerr=lcs[i].flux_err,color='r',fmt='o',label = f'{cam_names[i]} (pre-clip)',zorder=-10)
            plt.errorbar(lc.time.value,lc.flux,yerr=lc.flux_err,color='k',fmt='o',label = f'{cam_names[i]} (post-clip)',zorder=10) 
            plt.plot(hires_model_times,hires_optimized_model,label='hires optimized model',zorder=-15,alpha=1,color='purple') #plot the initial models
            plt.xlim(lc.time.value[0]-5,lc.time.value[-1]+5)
            plt.axvline(T0,label='T0')
            plt.ylim(np.min(lc.flux)-0.05,np.max(lc.flux)+0.05)
            plt.legend(loc='upper right')
            plt.show()
            
            np.save(f'../data/LCO_Photometry/{visit}_{this_filter}_{cam_names[i]}_times.npy',lc.time.value)
            np.save(f'../data/LCO_Photometry/{visit}_{this_filter}_{cam_names[i]}_fluxes.npy',lc.flux.value)
            np.save(f'../data/LCO_Photometry/{visit}_{this_filter}_{cam_names[i]}_uncertainties.npy',lc.flux_err.value)
    
            i += 1